In [1]:
import pandas as pd
from math import radians, cos, sqrt

In [2]:
def udaljenost_izmedju_koordinata(koordinate1, koordinate2):
    R = 6371.0

    sirina1_rad = radians(koordinate1[0])
    duzina1_rad = radians(koordinate1[1])
    sirina2_rad = radians(koordinate2[0])
    duzina2_rad = radians(koordinate2[1])

    razlika_sirina = sirina2_rad - sirina1_rad
    razlika_duzina = duzina2_rad - duzina1_rad

    x = razlika_duzina * cos((sirina1_rad + sirina2_rad) / 2)
    y = razlika_sirina
    distanca = sqrt(x**2 + y**2) * R

    return distanca

In [18]:
import ast
df = pd.read_csv("stajalista_sredjeno.csv")

lista_listi = df['Lista stanica'].apply(lambda x: ast.literal_eval(x))


distanca = udaljenost_izmedju_koordinata(lista_listi[0][0], lista_listi[0][1])

print(f"Distanca: {distanca} km.")

Distanca: 5.817834994076896 km.


In [6]:
dfTackica = pd.read_csv("linije_sredjeno.csv")

lista_tackica = dfTackica['Lista skretanja'].apply(lambda x: ast.literal_eval(x))


In [7]:
cnt = 0
mapiranje = {}
for line in lista_listi:
    tackice = lista_tackica[cnt]
    for stanica in line:
        min_dist = 100000000
        min_tacka = None
        for tacka in tackice:
            dist = udaljenost_izmedju_koordinata(stanica, tacka)
            if dist < min_dist:
                min_dist = dist
                min_tacka = tacka
        mapiranje[min_tacka] = stanica
    cnt += 1

print(mapiranje)

{(45.290610800313104, 19.82423633337021): (45.2916390663, 19.8237880279), (45.24277119952197, 19.847517907619476): (45.2420582997, 19.8475310279), (45.295180843019764, 19.822648465633392): (45.295793932, 19.8223251106), (45.29973919750302, 19.8227396607399): (45.2997652739, 19.8226695053), (45.27021660187992, 19.831344187259674): (45.2713036839, 19.8308548777), (45.26868004016919, 19.831875264644623): (45.2687779267, 19.8317437287), (45.23860489453414, 19.847855865955353): (45.2386999087, 19.84777609), (45.28243588315367, 19.8272243142128): (45.2828585422, 19.8268347377), (45.262676857415535, 19.83969658613205): (45.2623699914, 19.8401023551), (45.30263703365396, 19.824762046337128): (45.3025717506, 19.8247023774), (45.26564077226913, 19.835592806339264): (45.2656954001, 19.8354809218), (45.26100415945526, 19.842003285884857): (45.2609612857, 19.8420940143), (45.252711683009835, 19.84739989042282): (45.2524025428, 19.8471827488), (45.25558926422081, 19.841499030590057): (45.255203707, 

In [10]:
lista_stajalista_u_poretku = {}
lista_linija = df['ID linije'].tolist()
cnt = 0
for linija in lista_tackica:
    lista_stajalista_u_poretku[lista_linija[cnt]] = []
    for tacka in linija:
        if tacka in mapiranje:
            lista_stajalista_u_poretku[lista_linija[cnt]].append(mapiranje[tacka])
    cnt += 1


In [11]:
print(lista_stajalista_u_poretku['1A'])


[(45.3025717506, 19.8247023774), (45.2997652739, 19.8226695053), (45.295793932, 19.8223251106), (45.2916390663, 19.8237880279), (45.2888681125, 19.8247619009), (45.28628095, 19.8256640401), (45.2828585422, 19.8268347377), (45.2713036839, 19.8308548777), (45.2687779267, 19.8317437287), (45.2656954001, 19.8354809218), (45.2623699914, 19.8401023551), (45.2609612857, 19.8420940143), (45.255203707, 19.8416504854), (45.253337, 19.844187), (45.2524025428, 19.8471827488), (45.2474414598, 19.8488525074), (45.2420582997, 19.8475310279), (45.2386999087, 19.84777609)]


In [19]:
def promeni_stajalista(red):
    red['Lista stanica'] = str(lista_stajalista_u_poretku[red['ID linije']])
    return red
df.apply(lambda x: promeni_stajalista(x), axis=1)

,ID linije,Lista stanica
0,1A,"[(45.3025717506, 19.8247023774), (45.299765273..."
1,1B,"[(45.239001, 19.848256), (45.2429842112, 19.84..."
2,1ZA,"[(45.30128, 19.807564), (45.301786862, 19.8140..."
3,1ZB,"[(45.239001, 19.848256), (45.2429842112, 19.84..."
4,2A,"[(45.255083741, 19.8416138869), (45.2515776887..."
...,...,...
124,81B,"[(45.2116633546, 19.6068188465), (45.213760017..."
125,84A,"[(45.2640888897, 19.8285457138), (45.251451444..."
126,84B,"[(45.181009891, 19.5454098161), (45.1862903108..."
127,86A,"[(45.2640888897, 19.8285457138), (45.260177753..."


In [20]:
df.to_csv("stajalista_u_poretku.csv", index=False)